# U.S. Medical Insurance Costs

In [22]:
import csv
import sqlite3

# create sqlite database and make a connection to it
conn = sqlite3.connect('insurance.db')
cursor = conn.cursor()

# make the table in sql database
cursor.execute('''
    CREATE TABLE IF NOT EXISTS insurance (
        age INTEGER,
        sex TEXT,
        bmi REAL,
        children INTEGER,
        smoker TEXT,
        region TEXT,
        charges REAL
    )
''')

# read CSV and insert data
with open('insurance.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    for row in csv_reader:
        cursor.execute('''
            INSERT INTO insurance (age, sex, bmi, children, smoker, region, charges)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', row)

# commit transaction 
conn.commit()

# let's get some insights from this dataset
cursor.execute('''
    SELECT MIN(age) FROM insurance
''')
result = cursor.fetchall()
print("Minimum age:", result[0][0])

cursor.execute('''
    SELECT MAX(age) FROM insurance
''')
result = cursor.fetchall()
print("Maximum age:", result[0][0])
print("")


cursor.execute('SELECT ROUND(AVG(charges), 2) FROM insurance')
result = cursor.fetchall()

print("Average charges for all adults:")
print("Average charges: $", result[0][0])
print("")
      
cursor.execute('''
                SELECT age_ranges, ROUND(AVG(charges), 2) FROM (
                    SELECT charges, 
                    CASE
                        WHEN age < 20 THEN '18-19'
                        WHEN age >= 20 AND age < 30 THEN '20s'
                        WHEN age >= 30 AND age < 40 THEN '30s'
                        WHEN age >= 40 AND age < 50 THEN '40s'
                        WHEN age >= 50 AND age < 60 THEN '50s'
                        WHEN age >= 60 AND age < 70 THEN '60s'
                    END AS age_ranges
                    FROM insurance
                )
                GROUP BY age_ranges
                ''')
result = cursor.fetchall()
print("Average charges by age group")
for row in result:
    print("For people in their", row[0],": $", row[1])
print("")

cursor.execute('''
    SELECT sex, ROUND(AVG(charges), 2)
    FROM insurance
    GROUP BY sex
''')
result = cursor.fetchall()
print("Average charges by sex")
for row in result:
    sex = row[0]
    if sex == 'female':
        print("For women: $", row[1])
    else:
        print("For men: $", row[1])
print("")

cursor.execute("SELECT MIN(bmi) FROM insurance")
result = cursor.fetchall()
print("Minimum BMI:", result[0][0])

cursor.execute("SELECT MAX(bmi) FROM insurance")
result = cursor.fetchall()
print("Maximum BMI:", result[0][0])
print("")

cursor.execute('''
    SELECT bmi_ranges, ROUND(AVG(charges), 2) FROM (
        SELECT bmi, charges,
            CASE
                WHEN bmi >= 15 and bmi < 25 THEN "15-24.99"
                WHEN bmi >= 25 and bmi < 35 THEN "25-34.99"
                WHEN bmi >= 35 and bmi < 45 THEN "35-44.99"
                WHEN bmi >= 45 THEN "45+"
            END AS bmi_ranges
        FROM insurance
    )
    GROUP BY bmi_ranges
''')

result = cursor.fetchall()
print("Average charges by BMI group")
for row in result:
    print("For the", row[0], "BMI group: $", row[1])
print("")

cursor.execute('''
    SELECT children, ROUND(AVG(charges), 2)
    FROM insurance
    GROUP BY children
''')

result = cursor.fetchall()
print("Average amount of charges by number of kids")
for row in result:
    print("For people with", row[0], "kid(s): $", row[1])
print("")

cursor.execute('''
    SELECT smoker, ROUND(AVG(charges), 2)
    FROM insurance
    GROUP BY smoker
''')

result = cursor.fetchall()
print("Average charges by whether or not they smoke")
print("For non-smokers: $", result[0][1])
print("For smokers: $", result[1][1])
print("")

cursor.execute('''
    SELECT region, ROUND(AVG(charges), 2)
    FROM insurance
    GROUP BY region
''')
result = cursor.fetchall()
print("Average charges by region")
for row in result:
    print("For adults in the", row[0], ": $", row[1])


cursor.close()
conn.close()


Minimum age: 18
Maximum age: 64

Average charges for all adults:
Average charges: $ 13270.42

Average charges by age group
For people in their 18-19 : $ 8407.35
For people in their 20s : $ 9561.75
For people in their 30s : $ 11738.78
For people in their 40s : $ 14399.2
For people in their 50s : $ 16495.23
For people in their 60s : $ 21248.02

Average charges by sex
For women: $ 12569.58
For men: $ 13956.75

Minimum BMI: 15.96
Maximum BMI: 53.13

Average charges by BMI group
For the 15-24.99 BMI group: $ 10282.22
For the 25-34.99 BMI group: $ 12714.64
For the 35-44.99 BMI group: $ 16913.68
For the 45+ BMI group: $ 17547.93

Average amount of charges by number of kids
For people with 0 kid(s): $ 12365.98
For people with 1 kid(s): $ 12731.17
For people with 2 kid(s): $ 15073.56
For people with 3 kid(s): $ 15355.32
For people with 4 kid(s): $ 13850.66
For people with 5 kid(s): $ 8786.04

Average charges by whether or not they smoke
For non-smokers: $ 8434.27
For smokers: $ 32050.23

Averag